In [ ]:
from nlputils import utils
from nlputils.components import axaserver
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import json
import os

In [ ]:
# Define the path var to dir which need to be processed (including sub-dir)
# Ex:'../abc/project/
path_to_orig_data_folder = "folder to be considered/"

# Define file types to be considered
file_types = ['pdf','docx']

# read all files (inlcuding sub-dir) in dir defined above
files_list = utils.get_files(path_to_orig_data_folder)
print('individuals file type counts', '\n'.join([str(type)+":"+str(len(lst)) 
                                    for type,lst in files_list.items()]), sep='\n')

# Contructing dataframe for record keeping
# create list of key, values for dataframe
keys = [k for k, v in files_list.items() for _ in v]
values = [v for v in files_list.values() for v in v]
df_files = pd.DataFrame({'filetype': keys, 'orig_filepath': values})

individuals file type counts
pdf:430
docx:276


Convert docx files to pdf

In [ ]:
# Convert docx files to pdf
# converting docx to pdf is not deterministic as this might open a pop-up to grant 
# the permission which docx2pdf needs
converted_docx = utils.convert_docxfiles(files_list['docx'])
print('Docx succesfully converted:', sum(x is not None for x in converted_docx))
print('Docx not succesfully converted:',sum(x is None for x in converted_docx) )
# update the df with new path
tmp = dict(zip(files_list['docx'], converted_docx))
df_files['converted_filepath'] = df_files.orig_filepath.apply(lambda x: tmp.get(x))
df_files['page_count'] = df_files.orig_filepath.apply(lambda x: utils.get_page_count(x))

In [ ]:
# check if pdf is normal or imagepdf
df_files['ImagePDF'] = df_files.progress_apply(lambda x: utils.check_if_imagepdf(x['orig_filepath']) 
                                                if x['filetype']  == 'pdf' else
                                                (utils.check_if_imagepdf(x['converted_filepath']) 
                                                 if x['converted_filepath'] is not None else None),
                                                 axis=1)

100%|██████████| 706/706 [01:03<00:00, 11.09it/s]


In [ ]:
# save the files info
path_to_new_folder = "path to folder: Ex:'../abc/project/"
if not os.path.exists(path_to_new_folder + 'tmp/'):
            os.makedirs(path_to_new_folder + 'tmp/')
jsonfile = df_files.to_json(orient="records")
parsed = json.loads(jsonfile)
with open(path_to_new_folder+'tmp/df_files.json', 'w') as file:
    json.dump(parsed, file, indent=4)